# Fine-tune Prophetnet on Xsum

##Libraries and environment preparation

In [ ]:
#Install essential packages
%%capture
!pip install datasets transformers rouge-score nltk wandb
!apt install git-lfs

In [ ]:
#Colab Environment Check for GPU and RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

#GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Your runtime has 13.6 gigabytes of available RAM

Tue Feb 22 18:18:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                      

In [ ]:
# Make sure your version of Transformers is at least 4.11.0 
# to run the following code correctly:
import datasets
import transformers
print(transformers.__version__)

4.16.2


In [ ]:
from transformers import AutoTokenizer    
# Huggingface Automodel class
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_checkpoint = "microsoft/prophetnet-large-uncased"

In [ ]:
# Import Wandb 
import os
import wandb
API_KEY = '39991c538626bee25c64d4f8a4c3403dd635537c'
os.environ["WANDB_API_KEY"] = API_KEY

##Load the dataset

In [ ]:
# import dataset
raw_datasets = datasets.load_dataset("xsum")

Downloading:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/954 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

##Preprocess the data

In [ ]:
# Import tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/141 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:354: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [ ]:
# tokenlize inputs into map

max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_train = raw_datasets["train"].select(range(100000)).map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["document", "summary", "id"])

  0%|          | 0/100 [00:00<?, ?ba/s]

In [ ]:
tokenized_val = raw_datasets["validation"].select(range(5000)).map(preprocess_function, 
                                      batched=True,
                                      remove_columns=["document", "summary", "id"])

  0%|          | 0/5 [00:00<?, ?ba/s]

In [ ]:
tokenized_train

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 100000
})

##Fine-tuning the model

In [ ]:
# Import tokenizer from model checkpoint and print detail
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:354: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


Downloading:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

In [ ]:
wandb.init(project="prophetnet")

wandb: Currently logged in as: shusunny (use `wandb login --relogin` to force relogin)


In [ ]:
# Define traing args, batch size and epoch
# batch size max 8 for input length 1024 on Colab Pro

batch_size = 4
epochs = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    load_best_model_at_end="eval_loss",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=1000, 
    save_steps=2500,
    eval_steps=2500,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
)

PyTorch: setting up devices


In [ ]:
import nltk
import numpy as np
nltk.download('punkt')

metric = datasets.load_metric("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

In [ ]:
# Pass into the trainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using amp half precision backend


In [25]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `ProphetNetForConditionalGeneration.forward` and have been ignored: token_type_ids.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 100000
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 25000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
2500,3.130900,2.831381,35.247000,12.813400,27.844200,27.886200,24.872600
5000,2.929600,2.748818,36.362200,13.528700,28.650400,28.708000,25.668800
7500,2.870100,2.678171,37.128500,14.405900,29.459300,29.503800,25.816400
10000,2.805200,2.608081,37.799500,15.059500,30.057300,30.098300,26.020400
12500,2.760900,2.546456,38.344800,15.593500,30.644200,30.695200,26.255400
15000,2.670500,2.498819,38.871100,15.865600,30.953600,31.007600,25.909800


Streaming output truncated to the last 5000 lines.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with g

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
2500,3.130900,2.831381,35.247000,12.813400,27.844200,27.886200,24.872600
5000,2.929600,2.748818,36.362200,13.528700,28.650400,28.708000,25.668800
7500,2.870100,2.678171,37.128500,14.405900,29.459300,29.503800,25.816400
10000,2.805200,2.608081,37.799500,15.059500,30.057300,30.098300,26.020400
12500,2.760900,2.546456,38.344800,15.593500,30.644200,30.695200,26.255400
15000,2.670500,2.498819,38.871100,15.865600,30.953600,31.007600,25.909800
17500,2.638400,2.443130,39.410800,16.587700,31.602700,31.658500,26.179400
20000,2.582300,2.399606,40.237400,17.194900,32.348900,32.408200,25.941200
22500,2.541000,2.359188,40.781200,17.731000,32.852000,32.898000,26.489200
25000,2.475800,2.338430,41.073600,17.981400,33.056500,33.089500,26.221200


Streaming output truncated to the last 5000 lines.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`use_cache=True` is incompatible with g

TrainOutput(global_step=25000, training_loss=2.756292294921875, metrics={'train_runtime': 39834.2241, 'train_samples_per_second': 2.51, 'train_steps_per_second': 0.628, 'total_flos': 1.0582492322906112e+17, 'train_loss': 2.756292294921875, 'epoch': 1.0})

In [26]:
wandb.finish()

eval/gen_len,▁▄▅▆▇▅▇▆█▇
eval/loss,█▇▆▅▄▃▂▂▁▁
eval/rouge1,▁▂▃▄▅▅▆▇██
eval/rouge2,▁▂▃▄▅▅▆▇██
eval/rougeL,▁▂▃▄▅▅▆▇██
eval/rougeLsum,▁▂▃▄▅▅▆▇██
eval/runtime,▁▃▃▄▆▇▇▅▇█
eval/samples_per_second,█▆▆▅▃▂▂▄▂▁
eval/steps_per_second,█▆▆▅▃▂▂▄▂▁
train/epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████


In [36]:
!ls -lh 

total 4.0G
drwx------ 5 root root 4.0K Feb 22 18:17 drive
drwxr-xr-x 5 root root 4.0K Feb 23 06:28 prophetnet-large-uncased-finetuned-xsum
-rw-r--r-- 1 root root 4.0G Feb 23 06:34 prophetnet-large-uncased-finetuned-xsum.zip
drwxr-xr-x 1 root root 4.0K Feb 18 14:33 sample_data
drwxr-xr-x 3 root root 4.0K Feb 22 18:43 wandb


In [35]:
!zip -r /content/prophetnet-large-uncased-finetuned-xsum.zip /content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/

  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/ (stored 0%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/special_tokens_map.json (deflated 37%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/pytorch_model.bin (deflated 7%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/optimizer.pt (deflated 9%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/config.json (deflated 60%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/trainer_state.json (deflated 80%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/rng_state.pth (deflated 27%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/tokenizer_config.json (deflated 40%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/checkpoint-25000/scheduler.pt (deflated 49%)
  adding: content/prophetnet-large-uncased-finetuned-xsum/c

In [37]:
!cp prophetnet-large-uncased-finetuned-xsum.zip '/content/drive/My Drive/weights/'

##Try with a smaller dataset

In [ ]:
# Select to get smaller dataset
small_train = raw_datasets['train'].select(list(range(0, 5000)))
small_val = raw_datasets['validation'].select(list(range(0, 500)))
small_train

In [ ]:
tokenized_train = small_train.map(preprocess_function, batched=True)
tokenized_val = small_val.map(preprocess_function, batched=True)
tokenized_train

In [ ]:
# Import original model in the cache
model_small = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# data collator: pad the inputs and labels during each batch to save space
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_small)

In [ ]:
# keep track with wandb
wandb.init(project="Prophetnet")

In [ ]:
# Define traing args, batch size and epoch
# batch size max 8 for input length 1024 on Colab Pro

batch_size = 8
epochs = 1
model_name = model_checkpoint.split("/")[-1]
args_small = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    load_best_model_at_end="eval_loss",
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=1000, 
    save_steps=2500,
    eval_steps=2500,
    save_total_limit=3,
    num_train_epochs=epochs,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",
)

In [ ]:
# Pass into the trainer

trainer_small = Seq2SeqTrainer(
    model_small,
    args_small,
    train_dataset=small_train,
    eval_dataset=small_val,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer_small.train()